In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import save_image

"""noise = torch.random([1, 64, 64, 3])
generated_image = generator(noise)
plt.imshow(generated_image[0, :, :, :])


img = cv2.imread('24088735_p0.jpg', cv2.IMREAD_UNCHANGED)
imgs = cv2.resize(img, dsize=(64, 64), interpolation=cv2.INTER_AREA)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
imgs = cv2.cvtColor(imgs, cv2.COLOR_BGR2RGB)
img = img.astype(np.float32) / 255.
imgs = imgs.astype(np.float32) / 255.


plt.imshow(sr[0] * 255.)
"""
"""
writer = SummaryWriter()
writer.add_graph(netG, inp)
writer.close()
"""

'\nwriter = SummaryWriter()\nwriter.add_graph(netG, inp)\nwriter.close()\n'

In [2]:
from data import create_dataloader, create_dataset
from models.loss import GANLoss
import models.discriminator_vgg_arch as SRGAN_arch
import models.RRDBNet_arch as RRDBNet_arch
from utils.util import tensor2img, save_img

In [3]:
device = torch.device('cuda')

In [4]:
netG = RRDBNet_arch.RRDBNet(in_nc=3, out_nc=3,
                            nf=64, nb=3).to(device)
netD = SRGAN_arch.NLayerDiscriminator(input_nc=3, ndf=3, n_layers=2).to(device)
netF = SRGAN_arch.VGGFeatureExtractor(feature_layer=34, use_bn=False,
                                        use_input_norm=True, device=device)
netF.eval()

VGGFeatureExtractor(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), str

In [5]:
netG.train()
netD.train()

NLayerDiscriminator(
  (model): Sequential(
    (0): Conv2d(3, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(3, 6, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(6, 12, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1), bias=False)
    (6): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(12, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  )
)

In [6]:
img = cv2.imread('78652707.png', cv2.IMREAD_UNCHANGED)
imgs = cv2.resize(img, dsize=(64, 64), interpolation=cv2.INTER_AREA)

In [7]:
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
imgs = cv2.cvtColor(imgs, cv2.COLOR_BGR2RGB)

In [8]:
inp = torch.from_numpy(imgs)
inp = inp.view([1, 3, 64, 64]) / 255.
inp = inp.to(device)
inp.shape

torch.Size([1, 3, 64, 64])

In [9]:
var_H = torch.from_numpy(img)
var_H = var_H.view([1, 3, 256, 256]) / 255.
var_H = var_H.to(device)
var_H.shape

torch.Size([1, 3, 256, 256])

In [10]:
out = netG(inp)

In [11]:
out.shape

torch.Size([1, 3, 256, 256])

In [10]:
cri_pix = nn.L1Loss().to(device)
cri_fea = nn.L1Loss().to(device)
cri_gan = GANLoss('ragan').to(device)

In [11]:
optim_params = []
for k, v in netG.named_parameters():
    if v.requires_grad:
        optim_params.append(v)

optimizer_G = torch.optim.Adam(optim_params,
                                lr=1e-4,
                                weight_decay=0,
                                betas=(0.9, 0.999))
optimizer_D = torch.optim.Adam(netD.parameters(),
                                lr=1e-4,
                                weight_decay=0,
                                betas=(0.9, 0.999))

In [26]:
writer = SummaryWriter()
writer.add_graph(netG, inp)

In [22]:
for i in range(10000):

    for p in netD.parameters():
        p.requires_grad = False

    optimizer_G.zero_grad()
    fake_H = netG(inp.detach())

    l_g_total = 0

    l_g_pix = cri_pix(fake_H, var_H) * 1e12
    l_g_total += l_g_pix

    real_fea = netF(var_H).detach()
    fake_fea = netF(fake_H)
    l_g_fea = cri_fea(fake_fea, real_fea) * 1e8
    l_g_total += l_g_fea

    pred_g_fake = netD(fake_H)
    pred_d_real = netD(var_H).detach()
    l_g_gan = (
            cri_gan(pred_d_real - torch.mean(pred_g_fake), False) +
            cri_gan(pred_g_fake - torch.mean(pred_d_real), True)) * 1e8
    l_g_total += l_g_gan

    l_g_total.backward()
    optimizer_G.step()

    for p in netD.parameters():
        p.requires_grad = True

    optimizer_D.zero_grad()
    pred_d_real = netD(var_H)
    pred_d_fake = netD(fake_H.detach())  # detach to avoid BP to G
    l_d_real = cri_gan(pred_d_real - torch.mean(pred_d_fake), True)
    l_d_fake = cri_gan(pred_d_fake - torch.mean(pred_d_real), False)
    l_d_total = (l_d_real + l_d_fake) / 2

    l_d_total.backward()
    optimizer_D.step()

    if i % 10 == 0:
        print(f"l_g_t : {l_g_total}    l_d_t : {l_d_total}")
        if i % 100 == 0:
            tens = fake_H[0].view([256, 256, 3]).cpu().detach().numpy() * 255.
            cv2.imwrite(f"out/{i}.png", cv2.cvtColor(tens, cv2.COLOR_BGR2RGB))

l_g_t : 34073004032.0    l_d_t : 0.000220846151933074
l_g_t : 107829542912.0    l_d_t : 0.0018465437460690737
l_g_t : 85520523264.0    l_d_t : 0.0004581306711770594
l_g_t : 72541184000.0    l_d_t : 0.000267098774202168
l_g_t : 65366560768.0    l_d_t : 0.000215785505133681
l_g_t : 63327047680.0    l_d_t : 0.00020833261078223586
l_g_t : 61059076096.0    l_d_t : 0.0001771763345459476
l_g_t : 59781775360.0    l_d_t : 0.00016031040286179632
l_g_t : 57799147520.0    l_d_t : 0.00014208725770004094
l_g_t : 57050529792.0    l_d_t : 0.00013908369874116033
l_g_t : 54954827776.0    l_d_t : 0.00014189988723956048
l_g_t : 53754458112.0    l_d_t : 0.0001461140636820346
l_g_t : 52971327488.0    l_d_t : 0.0001471108989790082
l_g_t : 51935059968.0    l_d_t : 0.00015074570546858013
l_g_t : 50982141952.0    l_d_t : 0.0001561206445330754
l_g_t : 51479371776.0    l_d_t : 0.0001709216448944062
l_g_t : 49025380352.0    l_d_t : 0.00018176238518208265
l_g_t : 48210182144.0    l_d_t : 0.00019900569168385118
l_g_

KeyboardInterrupt: 

In [12]:
netF = netF.to(device)

In [25]:
fake_H[0] * 255.

tensor([[[  4.2791,   6.8633,  13.0753,  ...,   9.7725,   9.7298,   9.0559],
         [  0.4855,  13.4271,  17.4239,  ...,  14.9398,  14.9401,  12.6277],
         [  0.1360,  14.1932,  18.2004,  ...,  15.3612,  11.5537,  10.2342],
         ...,
         [ 11.0124,  18.5033,  15.4770,  ...,   7.9086,  12.4288,   6.3960],
         [  9.8670,  17.6951,  17.8377,  ...,   3.1074,   9.0118,   3.5490],
         [  5.0207,  10.0068,  11.5677,  ...,   7.8625,  11.9420,   7.7162]],

        [[  8.1659,  18.0494,  20.9779,  ...,  17.8085,  10.7644,   8.8299],
         [ 15.7267,  23.8540,  25.6983,  ...,  26.3135,  19.9894,  14.0191],
         [ 23.4594,  37.5542,  36.3386,  ...,  31.0273,  25.7456,  13.4786],
         ...,
         [  6.0464,  19.6664,  23.9531,  ...,  27.0898,  18.3888,  15.5247],
         [  4.7302,  11.6125,  16.3872,  ...,  20.5382,  13.2433,  10.2043],
         [ -1.3926,   1.4880,   4.0119,  ...,  11.9410,  10.9479,   8.6934]],

        [[ -6.4771,  -9.5513,  -5.7836,  ...

In [13]:
inp = torch.cat([inp, inp])
var_H = torch.cat([var_H, var_H])

In [14]:
var_H.shape

torch.Size([2, 3, 256, 256])

In [41]:
tens = var_H[0].view([256, 256, 3]).cpu().detach().numpy() * 255.
cv2.imwrite(f"out/{10}.png", cv2.cvtColor(tens, cv2.COLOR_BGR2RGB))

True

In [22]:
tens = fake_H[0].view([256, 256, 3]).cpu().detach().numpy() * 255.
cv2.imwrite(f"out/{10000}.png", cv2.cvtColor(tens, cv2.COLOR_BGR2RGB))

True

array([147., 152., 148.], dtype=float32)

In [27]:
ten = var_H[0].view([256, 256, 3]).cpu().detach().numpy() * 255.


In [51]:
cri_pix(fake_H[0], var_H[0])

tensor(0.4226, device='cuda:0', grad_fn=<L1LossBackward>)

In [15]:
torch.cuda.empty_cache()